In [45]:
import pandas as pd
import numpy as np

In [23]:
filepath = 'D:\Project\jupyter\machinelearning\horse_colic.csv'

In [46]:
df = pd.read_csv('watermelon3_0alpha.csv')
#df['outcome']
dataMat = np.mat(df[['density', 'suger_ratio']].values)
labelMat = np.mat(df[['clas']].values)

In [25]:
def loadSimpData():
    datMat = np.matrix([[ 1. ,  2.1],
        [ 2. ,  1.1],
        [ 1.3,  1. ],
        [ 1. ,  1. ],
        [ 2. ,  1. ]])
    classLabels = [1.0, 1.0, -1.0, -1.0, 1.0]
    return datMat,classLabels
dataMat, labelMat = loadSimpData()

In [74]:
def stumpClassify(dataMatrix,dimen,threshVal,threshIneq):#just classify the data
    retArray = np.ones((np.shape(dataMatrix)[0],1))
    if threshIneq == 'lt':
        retArray[dataMatrix[:,dimen] <= threshVal] = -1.0
    else:
        retArray[dataMatrix[:,dimen] > threshVal] = -1.0
    return retArray

In [78]:
"""
input:
    D: 权重向量

"""
def buildStump(dataMatrix,labelMat,D):
    m,n = np.shape(dataMatrix)
    numSteps = 10.0; bestStump = {}; bestClasEst = np.mat(np.zeros((m,1)))
    minError = np.inf #init error sum, to +infinity
    for i in range(n):#loop over all dimensions
        rangeMin = dataMatrix[:,i].min(); rangeMax = dataMatrix[:,i].max();
        stepSize = (rangeMax-rangeMin)/numSteps
        for j in range(-1,int(numSteps)+1):#loop over all range in current dimension
            for inequal in ['lt', 'gt']: #go over less than and greater than
                threshVal = (rangeMin + float(j) * stepSize)
                predictedVals = stumpClassify(dataMatrix,i,threshVal,inequal)#call stump classify with i, j, lessThan
                errArr = np.mat(np.ones((m,1)))
                errArr[predictedVals == labelMat] = 0
                weightedError = D.T*errArr  #calc total error multiplied by D
                #print("split: dim %d, thresh %.2f, thresh ineqal: %s, the weighted error is %.3f" % (i, threshVal, inequal, weightedError))
                if weightedError < minError:
                    minError = weightedError
                    bestClasEst = predictedVals.copy()
                    bestStump['dim'] = i
                    bestStump['thresh'] = threshVal
                    bestStump['ineq'] = inequal
    return bestStump,minError,bestClasEst


In [100]:
"""
基于"重赋权法"的特定数据分布
input:
    dataMat:数据集
    T:训练轮数
output:
    H(x) = sign(求和alphat * ht(x))
"""
def adaBoostTrain(dataMat, labelMat, T):
    weakClassfier = []
    row, col = dataMat.shape
    D = np.mat(np.ones((row, 1)) / row)
    aggClassEst = np.mat(np.zeros((row, 1)))
    for i in range(T):
        bestStump, error, classEst = buildStump(dataMat, labelMat, D)
        if error > 0.5 :
            break
        alpha = float(0.5 * np.log((1 - error)/error))
        bestStump['alpha'] = alpha
        D = np.multiply(D, np.exp(-alpha * np.multiply(labelMat, classEst)))/D.sum()
        weakClassfier.append(bestStump)
        aggClassEst += alpha * classEst
        print('第{}个分类器 aggClassEst : {}'.format(i, aggClassEst))
        aggError = np.multiply((np.sign(aggClassEst) != labelMat), labelMat)
        errorRate = aggError.sum() / row
        print('第{}个分类器 errorRate: {}'.format(i, errorRate))
        
    return weakClassfier

In [66]:
def adaBoost(dataMat, weakClassfier):
    row, col = dataMat.shape
    aggClassEst = np.mat(np.zeros((row, 1)))
    for wCf in weakClassfier:
        classEst = stumpClassify(dataMat, wCf['dim'], wCf['threshVal'], wCf['threshIneq'])
        aggClassEst += wCf['alpha'] * classEst
        print('aggClassEst:', aggClassEst)
    return np.sign(aggClassEst)

In [101]:
adaBoostTrain(dataMat, labelMat, 5)

第0个分类器 aggClassEst : [[ 0.77022252]
 [ 0.77022252]
 [ 0.77022252]
 [ 0.77022252]
 [ 0.77022252]
 [ 0.77022252]
 [-0.77022252]
 [ 0.77022252]
 [-0.77022252]
 [ 0.77022252]
 [-0.77022252]
 [-0.77022252]
 [-0.77022252]
 [-0.77022252]
 [ 0.77022252]
 [-0.77022252]
 [-0.77022252]]
第0个分类器 errorRate: -0.058823529411764705
第1个分类器 aggClassEst : [[ 1.69404342]
 [ 1.69404342]
 [ 1.69404342]
 [ 1.69404342]
 [ 1.69404342]
 [ 1.69404342]
 [ 0.15359838]
 [ 1.69404342]
 [ 0.15359838]
 [-0.15359838]
 [-1.69404342]
 [-1.69404342]
 [ 0.15359838]
 [ 0.15359838]
 [-0.15359838]
 [ 0.15359838]
 [ 0.15359838]]
第1个分类器 errorRate: -0.29411764705882354
第2个分类器 aggClassEst : [[ 0.83158064]
 [ 2.5565062 ]
 [ 0.83158064]
 [ 0.83158064]
 [ 0.83158064]
 [ 0.83158064]
 [-0.7088644 ]
 [ 0.83158064]
 [-0.7088644 ]
 [-1.01606116]
 [-2.5565062 ]
 [-2.5565062 ]
 [-0.7088644 ]
 [-0.7088644 ]
 [-1.01606116]
 [-0.7088644 ]
 [-0.7088644 ]]
第2个分类器 errorRate: 0.058823529411764705
第3个分类器 aggClassEst : [[ 1.62483038]
 [ 3.34975594]


[{'alpha': 0.7702225204735745,
  'dim': 1,
  'ineq': 'lt',
  'thresh': 0.20920000000000002},
 {'alpha': 0.9238208993747057, 'dim': 0, 'ineq': 'lt', 'thresh': 0.4023},
 {'alpha': 0.8624627800932602, 'dim': 0, 'ineq': 'lt', 'thresh': 0.7209},
 {'alpha': 0.7932497413463995,
  'dim': 1,
  'ineq': 'lt',
  'thresh': 0.12560000000000002},
 {'alpha': 0.5749446067975488, 'dim': 1, 'ineq': 'lt', 'thresh': 0.3764}]